In [134]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from lxml import objectify
import collections
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import cross_validation
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical

from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# from theano.tensor.shared_randomstreams import RandomStreams
# fix random seed for reproducibility
np.random.seed(7)
#srng = RandomStreams(7)

In [2]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [56]:
path = 'calendartestdata.xml'
xml = objectify.parse(open(path))
root = xml.getroot()
objphrase = str(root.getchildren()[0]).split('|')
obj = str(root.getchildren()[1]).split('|')
obj[2] = obj[2].split('+') if '+' in obj[2] else [obj[2],0,0,0]
obj[4] = obj[4].split('+') if '+' in obj[4] else [obj[4],0,0,0]

In [52]:
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, basestring):
            for sub in flatten(el):
                yield sub
        else:
            yield el

In [57]:
obj = flatten(obj)
print(objphrase)
print(list(obj))
#len(root.getchildren())

['Grocery shopping at King Kullen Thursday at 11:30pm']
['Grocery shopping', 'King Kullen', 'thursday', 0, 0, 0, '11:30 PM', 'friday', 0, 0, 0, '12:30 AM', '0', 'none']


In [60]:
calTrainingSet = pd.DataFrame(columns=('phrase', 'summary', 'location', 'daysFromNowStart', 'weeksFromNowStart',
                                       'monthsFromNowStart', 'yearsFromNowStart', 'startHour', 'daysFromNowEnd',
                                       'weeksFromNowEnd', 'monthsFromNowEnd', 'yearsFromNowEnd',
                                       'endHour', 'allDayFlag', 'repeatFrequency'))

In [61]:
for i in range(0,len(root.getchildren()), 2):
    phrase = str(root.getchildren()[i]).split('|')
    parsed = str(root.getchildren()[i+1]).split('|')
    parsed[2] = parsed[2].split('+') if '+' in parsed[2] else [parsed[2],0,0,0]
    parsed[4] = parsed[4].split('+') if '+' in parsed[4] else [parsed[4],0,0,0]
    parsed = flatten(parsed)
    row = pd.Series(dict(zip(calTrainingSet.columns, phrase + list(parsed))))
    row.name = i
    calTrainingSet = calTrainingSet.append(row)

In [62]:
calTrainingSet.head()

,phrase,summary,location,daysFromNowStart,weeksFromNowStart,monthsFromNowStart,yearsFromNowStart,startHour,daysFromNowEnd,weeksFromNowEnd,monthsFromNowEnd,yearsFromNowEnd,endHour,allDayFlag,repeatFrequency
0,Grocery shopping at King Kullen Thursday at 11...,Grocery shopping,King Kullen,thursday,0,0,0,11:30 PM,friday,0,0,0,12:30 AM,0,none
2,Grocery shopping at Waldbaums from 4pm to 5pm,Grocery shopping,Waldbaums,,0,0,0,04:00 PM,,0,0,0,05:00 PM,0,none
4,Clothes shopping Wednesday at 5:35pm at Nordst...,Clothes shopping,Nordstroms,wednesday,0,0,0,05:35 PM,wednesday,0,0,0,06:35 PM,0,none
6,Clothes shopping at Nordstroms in a month at 1...,Clothes shopping,Nordstroms,0,0,1,0,05:45 PM,0,0,1,0,06:45 PM,0,none
8,Clothes shopping at Nordstroms next Thursday a...,Clothes shopping,Nordstroms,nextthursday,0,0,0,05:00 PM,nextthursday,0,0,0,06:00 PM,0,none


In [63]:
calTrainingSet.to_csv('calendartestdata.csv', index = False)

In [112]:
X = calTrainingSet.iloc[:,0]
y = calTrainingSet.iloc[:,1]
y.head()

0    Grocery shopping
2    Grocery shopping
4    Clothes shopping
6    Clothes shopping
8    Clothes shopping
Name: summary, dtype: object

In [108]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.2, random_state=42)

In [149]:
ngram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(2, 2), min_df=1)
X_train = ngram_vectorizer.fit_transform(X_train)
X_test = ngram_vectorizer.fit_transform(X_test)
len(counts.toarray()[0])
# vectorizer = TfidfVectorizer(max_df=0.5,
#                                  min_df=2, stop_words='english')
# test = vectorizer.fit_transform(X_train)
# test.toarray().astype(int)[3]

461

In [152]:
# truncate and pad input sequences
max_review_length = 461
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

TypeError: sparse matrix length is ambiguous; use getnnz() or shape[0]

In [109]:
def one_hot_encode_object_array(arr):
    '''One hot encode a numpy array of objects (e.g. strings)'''
    uniques, ids = np.unique(arr, return_inverse=True)
    print(np.unique(arr))
    return to_categorical(ids, len(uniques))

In [115]:
y_train_ohe = one_hot_encode_object_array(y_train)
y_test_ohe = one_hot_encode_object_array(y_test)
# for i in range(0, len(y_train)):
#     if i == 0:
#         y_train_ohe = one_hot_encode_object_array(y_train.iloc[:,i])
#     else:
#         y_train_ohe = np.concatenate([y_train_ohe, one_hot_encode_object_array(y_train.iloc[:,i])], axis = 1)
# for i in range(0, len(y_test)):
#     if i == 0:
#         y_test_ohe = one_hot_encode_object_array(y_test.iloc[:,i])
#     else:
#         y_test_ohe = np.concatenate([y_test_ohe, one_hot_encode_object_array(y_test.iloc[:,i])], axis = 1)

['' 'Bake a cake' 'Bank holiday' 'Baseball Game' 'Birthday party'
 "Brendan's 6th birthday party" "Brunch for Auntie's birthday" 'Buy milk'
 'Call John Bates to apologize' 'Call John Bates with apology' 'Christmas'
 'Clothes shopping' 'Conference call' 'Dinner' 'Dinner with Ben Kerwing'
 'Dinner with Rhiannon' 'Dinner with Rocko' 'Exam' 'Family vacation'
 'Finish important task' 'Finish working on the platter' 'Fondue'
 'Football Game' 'Graduate Computer Graphics' 'Grocery shopping'
 'High school admission results' 'Homework 5 due' 'I work' 'Jogging'
 'Language Class' "Let's have lunch" 'Lunch with English Project Team'
 'Lunch with John' 'Lunch with Matthew' 'Lunch with Preshit'
 'Lunch with becca' 'Meet with Cody' 'Meeting'
 'Meeting with Johnny, Marco, and Alexandri to talk about the research'
 "Mom's birthday" "My mom's birthday" 'Paper due' 'Party' 'Pick up Joe'
 'Picnic' 'Project meeting with Jason' 'Retreat' 'Running w/ Pat'
 'Shopping' 'Soccer practice' 'Staff meeting' 'Star ga

In [116]:
y_train_ohe.shape

(112, 76)

In [100]:
model = Sequential()
model.add(Dense(16, input_shape=(290,)))
model.add(Activation("relu"))
model.add(Dense(290))
model.add(Activation("softmax"))

In [105]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [106]:
model.fit(X_train, y_train_ohe, nb_epoch=5, batch_size=1)

Epoch 1/5


ValueError: could not convert string to float: Family vacation from August 9-18 at The Marriot Hotel